In [4]:
from pyscf import gto,scf
import pandas as pd
from pyscf.hessian import rhf as hrhf
import numpy as np 
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
import basis_set_exchange as bse
import sys
dl=.05
sys.path.append('/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/')
from FDcoeffs import *
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM

In [5]:
#Doing []="at the geom of"
#NH4[CH4] , BH4[CH4]
# BF[CO],CO[BF],NN[CO],CO[NN]

In [6]:
l=3.3376
x=-(1/24)**.5*l
z=(2/3)**.5*l+x
nh4str="N 0 0 0; H 0 0  {}; H  0  {} {}; H {} {} {}; H {} {} {}".format(z,l/3**.5,x,l/2,-l*3**.5/6,x,-l/2,-l*3**.5/6,x)
bh4str="B 0 0 0; H 0 0  {}; H  0  {} {}; H {} {} {}; H {} {} {}".format(z,l/3**.5,x,l/2,-l*3**.5/6,x,-l/2,-l*3**.5/6,x)

In [19]:
BH4=gto.M(atom=bh4str,unit="bohr",basis={"H":"pc-2",'B':bse.get_basis("pcX-2",fmt="nwchem",elements=[5])},charge=-1)
NH4=gto.M(atom=nh4str,unit="bohr",basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])},charge=1)

In [12]:
BFatCO=gto.M(atom="B 0 0 0; F 0 0 2.08272",unit="Bohrs",basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]))
NNatCO=gto.M(atom="N 0 0 0; N 0 0 2.08272",unit="Bohrs",basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]))

In [13]:
COatBF=gto.M(atom="C 0 0 0; O 0 0 2.35348",unit="Bohrs",basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]))
COatNN=gto.M(atom="C 0 0 0; O 0 0 2.01388999",unit="Bohrs",basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]))

In [27]:
q=[]

In [28]:
for x in ["BH4","NH4","BFatCO","NNatCO","COatBF","COatNN"]:
    exec("mf=scf.RHF({})".format(x))
    mf.scf()
    g=mf.Gradients().grad()
    h=mf.Hessian().hess()
    q.append({"molecule":x,"gradient":g[1,2],"hessian":h[1,1,2,2]})
q2=q

converged SCF energy = -26.9401976192551
--------------- RHF gradients ---------------
         x                y                z
0 B    -0.0000000000     0.0000000000    -0.0000000000
1 H     0.0000000000    -0.0000000000    -0.0903647176
2 H    -0.0000000000    -0.0851966728     0.0301215725
3 H    -0.0737824829     0.0425983364     0.0301215725
4 H     0.0737824829     0.0425983364     0.0301215725
----------------------------------------------
converged SCF energy = -56.5511714490662
--------------- RHF gradients ---------------
         x                y                z
0 N     0.0000000000     0.0000000000    -0.0000000000
1 H     0.0000000000     0.0000000000     0.0536377667
2 H     0.0000000000     0.0505701714    -0.0178792556
3 H     0.0437950531    -0.0252850857    -0.0178792556
4 H    -0.0437950531    -0.0252850857    -0.0178792556
----------------------------------------------
converged SCF energy = -124.131780705027
--------------- RHF gradients ---------------
     

In [29]:
q2

[{'molecule': 'BH4',
  'gradient': -0.09036471758108888,
  'hessian': 0.4033736521752618},
 {'molecule': 'NH4',
  'gradient': 0.05363776666594178,
  'hessian': 0.3031051665449356},
 {'molecule': 'BFatCO',
  'gradient': -0.26754828296754773,
  'hessian': 1.5143654274925282},
 {'molecule': 'NNatCO',
  'gradient': 0.12039897206164873,
  'hessian': 1.5389424031892336},
 {'molecule': 'COatBF',
  'gradient': 0.2599559115472605,
  'hessian': 0.5270850155450217},
 {'molecule': 'COatNN',
  'gradient': -0.11942017426455642,
  'hessian': 1.9530809144415002}]

In [30]:
df=pd.DataFrame(q2)
df

,gradient,hessian,molecule
0,-0.090365,0.403374,BH4
1,0.053638,0.303105,NH4
2,-0.267548,1.514365,BFatCO
3,0.120399,1.538942,NNatCO
4,0.259956,0.527085,COatBF
5,-0.119420,1.953081,COatNN


In [31]:
df.to_pickle("target_grad_hessian")